# One classifier each name: OCEN (Name group)
This method throw away the record below threshold.

Also this file evaluate different embedding method with OCEN.
1. Text embedding only
2. Citation embedding only
3. Combined embedding

In [1]:
import os
import sys
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import com_func

# parameters
#----- threshold for selecting set of name group -----------#
threshold_select_name_group = 100
#----- threshold for selecting min sample in name group ----#
threshold_min_class_sample_size = 100

Dataset = "pubmed"

In [2]:
# text embedding only
pp_text_emb = ["tf", "tf_idf", "lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["off"]

In [5]:
# citation embedding only
pp_text_emb = ["off"]
pp_citation_emb = ["node2vec", "n2v"]

In [2]:
# combined embedding
pp_text_emb = ["lsa", "pv_dm", "pv_dbow"]
pp_citation_emb = ["n2v"]

In [4]:
print(pp_text_emb)
print(pp_citation_emb)

['tf', 'tf_idf', 'lsa', 'pv_dm', 'pv_dbow']
['off']


In [5]:
import io
import collections
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from statistics import mean 

# fix random seed for reproducibility
np.random.seed(1)

fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

lr_diff_embedding_result = []
svm_diff_embedding_result = []

# ----------------------- different textual embedding ----------------------#
for text_emb in pp_text_emb:
    print("Load text embedding: ", text_emb)
    # read pretrained embeddings
    all_textual_embedding, all_textual_emb_pid = com_func.read_text_embedding(emb_type=text_emb, training_size="140k")
    
    for citation_emb in pp_citation_emb:
        print("Load citation embedding: ", citation_emb)
        all_citation_embedding = com_func.read_citation_embedding_sorted(emb_type = citation_emb)
        all_citation_emb_pid = [emb[0] for emb in all_citation_embedding]
        all_citation_embedding = [emb[1:] for emb in all_citation_embedding]
        
        # collect statistic to output
        allname, num_class, per_class_count, selected_sample_size, orginal_sample_size= ([] for i in range(5))
        all_mnb_accuracy, all_mnb_f1 = ([] for i in range(2))
        all_LR_accuracy, all_LR_f1 = ([] for i in range(2))
        all_svcLinear_accuracy, all_svcLinear_f1 = ([] for i in range(2))

        total_selected_group = 0

        # ------- different name group in all name group --------------------#
        for file in listfiles:
            # group name
            temp = file.split("_")
            name = temp[1]+"_"+temp[-1]
            print("For name: ",name)
            # read needed content in labeled file
            labeled_data = com_func.read_pid_aid(fileDir+file)
            #----------- select name group contain productive author------------------------------------#
            #----------- (contain pair of author write more than 100 papers) ---------------------------#
            # count number of paper each author write based on author ID
            authorCounter = collections.Counter(labeled_data["authorID"])
            # remove name group that do not contain pair of author write more than 100 papers
            for k in list(authorCounter):
                if authorCounter[k] < threshold_select_name_group:
                    del authorCounter[k]
            # if only have one class or no class pass the threshold, not applicable
            if(len(authorCounter)==0) or (len(authorCounter)==1):
                print(name, " pass")
            else:
                total_selected_group+= 1
                orginal_sample_size.append(len(labeled_data))
                #--------select authors in name group are very productive (more than threshold)---------#
                print("Total sample size before apply threshold: ",len(labeled_data))
                # count number of paper each author write based on author ID
                paperCounter = collections.Counter(labeled_data["authorID"])
                print(paperCounter)
                print("Total author before apply threshoid: ", len(paperCounter))
                # collect per class statistic
                for k in list(paperCounter):
                    if paperCounter[k] < threshold_min_class_sample_size:
                        del paperCounter[k]
                temp =list(paperCounter.keys())
                print(temp)
                print("Total author after apply threshoid: ", len(temp))
                # remove samples that are smaller than threshold
                labeled_data = labeled_data[labeled_data.authorID.isin(temp)]
                print("Total sample size after apply threshold: ",len(labeled_data))
                selected_sample_size.append(len(labeled_data))
                allname.append(name)
                num_class.append(len(paperCounter))
                per_class_count.append(paperCounter)
                #------------ extract paper representation -------------------------------------------#
                # shuffle the data
                labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
                # extract true label and pid
                label = labeled_data["authorID"]
                pid = labeled_data["paperID"]
                # list of different data field
                part_collection = []
                # select feature wanted to fit to clustering/classification algorithm
                # data part, textual information
                data_part_textual = com_func.extract_embedding(all_textual_embedding, all_textual_emb_pid, pid)
                print("Text embedding shape: ", data_part_textual.shape)
                part_collection.append(data_part_textual)
                # data part, citation information
                data_part_citation = com_func.extract_embedding(all_citation_embedding, all_citation_emb_pid, pid)
                print("Citation embedding shape: ", data_part_citation.shape)
                part_collection.append(data_part_citation)
                # merge different part of data data together by concatenate it all together
                # remove empty emb (when emb set off)
                part_collection = [part for part in part_collection if len(part)!=0]
                if len(part_collection)>1:
                    combinedata = np.concatenate(part_collection,axis=1)
                elif len(part_collection)==1:
                    if isinstance(part_collection[0], pd.DataFrame):
                        combinedata = part_collection[0].values
                    else:
                        combinedata = part_collection[0]
                else:
                    print("No data available")
                    break
                print("Final feature (combined embedding) shape: ", combinedata.shape)
                # -------------- using converted feature vector to train classifier-------------------#
                if text_emb == "tf":
                    # using multinomial naive bayes
                    clf = MultinomialNB()
                    mnbaccuracy, mnbmarcof1, tp, tn, fp, fn = com_func.k_fold_cv_with_accumulate_statistic(combinedata, label, clf, k=10)
                    print("MNB Accuracy: ",mnbaccuracy)
                    print("MNB F1: ", mnbmarcof1)
                    all_mnb_accuracy.append(mnbaccuracy)
                    all_mnb_f1.append(mnbmarcof1)
                # using logistic regression
                clf = LogisticRegression(multi_class='ovr')
                LRaccuracy, LRmarcof1, tp, tn, fp, fn = com_func.k_fold_cv_with_accumulate_statistic(combinedata, label, clf, k=10)
                print("LR Accuracy: ",LRaccuracy)
                print("LR F1: ", LRmarcof1)
                all_LR_accuracy.append(LRaccuracy)
                all_LR_f1.append(LRmarcof1)
                # using SVM with linear kernal
                clf = SVC(decision_function_shape='ovr', kernel='linear')
                svcaccuracy, svcmarcof1, tp, tn, fp, fn = com_func.k_fold_cv_with_accumulate_statistic(combinedata, label, clf, k=10)
                print("svc Accuracy: ",svcaccuracy)
                print("svc F1: ", svcmarcof1)
                all_svcLinear_accuracy.append(svcaccuracy)
                all_svcLinear_f1.append(svcmarcof1)

        if text_emb == "tf":
            # write evaluation result to excel
            output = pd.DataFrame({'Name Group':allname,"Class number":num_class,"Per class size":per_class_count, 
                                   "Total selected sample size":selected_sample_size,"Orginal sample size":orginal_sample_size,
                                   "MNB Accuracy":all_mnb_accuracy, "MNB macro F1": all_mnb_f1, 
                                   "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                                   "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})
        else:
            # write evaluation result to excel
            output = pd.DataFrame({'Name Group':allname, "Class number":num_class, "Per class size":per_class_count,
                                   "Total selected sample size":selected_sample_size,"Orginal sample size":orginal_sample_size,
                                   "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                                   "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})

        savePath = "../../result/"+Dataset+"/OCEN_sample=140k/"
        filename = "citation="+citation_emb+"_textual="+text_emb+"_threshold="+str(threshold_min_class_sample_size)+".csv"
        com_func.write_csv_df(savePath, filename, output)
        print("Done")

        lr_diff_embedding_result.append(all_LR_f1)
        svm_diff_embedding_result.append(all_svcLinear_f1)

Load text embedding:  tf
Total text vector records: 135796
Vector dimension:  50000
Load citation embedding:  off
For name:  j_read
j_read  pass
For name:  f_esteves
f_esteves  pass
For name:  c_miller
c_miller  pass
For name:  r_jha
r_jha  pass
For name:  a_lowe
a_lowe  pass
For name:  a_vega
a_vega  pass
For name:  k_smith
k_smith  pass
For name:  j_gordon
j_gordon  pass
For name:  s_liao
s_liao  pass
For name:  j_qian
j_qian  pass
For name:  s_bernardi
s_bernardi  pass
For name:  t_hill
t_hill  pass
For name:  s_schindler
s_schindler  pass
For name:  j_williams
j_williams  pass
For name:  s_jacobson
s_jacobson  pass
For name:  e_andrade
e_andrade  pass
For name:  t_santos
t_santos  pass
For name:  k_kim
Total sample size before apply threshold:  1111
Counter({'0000-0002-6929-5359': 211, '0000-0001-9498-284X': 154, '0000-0002-5878-8895': 139, '0000-0002-1864-3392': 92, '0000-0002-7045-8004': 57, '0000-0001-7896-6751': 57, '0000-0002-7991-9428': 55, '0000-0002-4010-1063': 45, '0000-00

a_nielsen  pass
For name:  j_conde
j_conde  pass
For name:  k_wright
k_wright  pass
For name:  m_parker
m_parker  pass
For name:  h_huang
h_huang  pass
For name:  j_terry
j_terry  pass
For name:  y_xu
y_xu  pass
For name:  a_melo
a_melo  pass
For name:  r_doyle
r_doyle  pass
For name:  m_bernardo
m_bernardo  pass
For name:  j_soares
j_soares  pass
For name:  j_richard
j_richard  pass
For name:  p_robinson
Total sample size before apply threshold:  275
Counter({'0000-0002-7878-0313': 133, '0000-0002-0736-9199': 119, '0000-0002-3156-3418': 19, '0000-0002-0577-3147': 4})
Total author before apply threshoid:  4
['0000-0002-0736-9199', '0000-0002-7878-0313']
Total author after apply threshoid:  2
Total sample size after apply threshold:  252
Total missing sample:  0
Text embedding shape:  (252, 50000)
Total missing sample:  0
Citation embedding shape:  (0, 0)
Final feature (combined embedding) shape:  (252, 50000)
Binary case
                     precision    recall  f1-score   support

000

c_nogueira  pass
For name:  j_cooper
j_cooper  pass
For name:  k_lau
k_lau  pass
For name:  s_hussein
s_hussein  pass
For name:  z_luo
z_luo  pass
For name:  c_pimentel
c_pimentel  pass
For name:  s_ito
s_ito  pass
For name:  f_zhang
f_zhang  pass
For name:  s_chapman
s_chapman  pass
For name:  j_rosa
j_rosa  pass
For name:  y_yin
y_yin  pass
For name:  p_tavares
p_tavares  pass
For name:  a_palma
a_palma  pass
For name:  e_shaw
e_shaw  pass
For name:  m_cameron
m_cameron  pass
For name:  a_reid
a_reid  pass
For name:  d_gil
d_gil  pass
For name:  s_morgan
s_morgan  pass
For name:  p_ross
p_ross  pass
For name:  l_simon
l_simon  pass
For name:  k_thomas
k_thomas  pass
For name:  l_torres
l_torres  pass
For name:  p_ding
p_ding  pass
For name:  g_morris
g_morris  pass
For name:  s_andrews
s_andrews  pass
For name:  b_yan
b_yan  pass
For name:  r_hu
r_hu  pass
For name:  j_braun
j_braun  pass
For name:  c_he
c_he  pass
For name:  w_lu
w_lu  pass
For name:  r_radhakrishnan
r_radhakrishnan

Total missing sample:  0
Text embedding shape:  (208, 50000)
Total missing sample:  0
Citation embedding shape:  (0, 0)
Final feature (combined embedding) shape:  (208, 50000)
Binary case
                     precision    recall  f1-score   support

0000-0001-5833-989X       0.95      0.99      0.97       100
0000-0003-3171-7672       0.99      0.95      0.97       108

           accuracy                           0.97       208
          macro avg       0.97      0.97      0.97       208
       weighted avg       0.97      0.97      0.97       208

TP:  103 TN:  99 FP:  1 FN:  5
MNB Accuracy:  0.9711538461538461
MNB F1:  0.9711431742508324
Binary case
                     precision    recall  f1-score   support

0000-0001-5833-989X       1.00      0.98      0.99       100
0000-0003-3171-7672       0.98      1.00      0.99       108

           accuracy                           0.99       208
          macro avg       0.99      0.99      0.99       208
       weighted avg       0.99 

Binary case
                     precision    recall  f1-score   support

0000-0002-6391-1341       1.00      0.95      0.97       112
0000-0002-6794-6656       0.97      1.00      0.98       180

           accuracy                           0.98       292
          macro avg       0.98      0.97      0.98       292
       weighted avg       0.98      0.98      0.98       292

TP:  180 TN:  106 FP:  6 FN:  0
svc Accuracy:  0.9794520547945206
svc F1:  0.9780418107986163
For name:  r_sinha
r_sinha  pass
For name:  c_turner
c_turner  pass
For name:  y_su
y_su  pass
For name:  a_popov
a_popov  pass
For name:  w_liao
w_liao  pass
For name:  j_zhong
j_zhong  pass
For name:  a_wheeler
a_wheeler  pass
For name:  m_walsh
m_walsh  pass
For name:  r_figueiredo
r_figueiredo  pass
For name:  y_lin
Total sample size before apply threshold:  785
Counter({'0000-0003-3791-7587': 146, '0000-0001-8153-1441': 115, '0000-0003-1224-6561': 64, '0000-0002-4192-3165': 49, '0000-0002-2499-8632': 39, '0000-0001

...

Total missing sample:  0
Text embedding shape:  (314, 100)
Total missing sample:  0
Citation embedding shape:  (0, 0)
Final feature (combined embedding) shape:  (314, 100)
Binary case
                     precision    recall  f1-score   support

0000-0001-5525-0494       0.99      0.99      0.99       156
0000-0003-2861-8286       0.99      0.99      0.99       158

           accuracy                           0.99       314
          macro avg       0.99      0.99      0.99       314
       weighted avg       0.99      0.99      0.99       314

TP:  156 TN:  154 FP:  2 FN:  2
LR Accuracy:  0.9872611464968153
LR F1:  0.987260629665693
Binary case
                     precision    recall  f1-score   support

0000-0001-5525-0494       0.98      0.98      0.98       156
0000-0003-2861-8286       0.98      0.98      0.98       158

           accuracy                           0.98       314
          macro avg       0.98      0.98      0.98       314
       weighted avg       0.98      0

c_yang  pass
For name:  s_paul
s_paul  pass
For name:  l_roberts
Total sample size before apply threshold:  363
Counter({'0000-0003-4270-253X': 206, '0000-0001-7885-8574': 120, '0000-0002-1455-5248': 18, '0000-0003-0085-9213': 14, '0000-0003-3892-2900': 3, '0000-0002-0329-8389': 2})
Total author before apply threshoid:  6
['0000-0003-4270-253X', '0000-0001-7885-8574']
Total author after apply threshoid:  2
Total sample size after apply threshold:  326
Total missing sample:  0
Text embedding shape:  (326, 100)
Total missing sample:  0
Citation embedding shape:  (0, 0)
Final feature (combined embedding) shape:  (326, 100)
Binary case
                     precision    recall  f1-score   support

0000-0001-7885-8574       1.00      0.95      0.97       120
0000-0003-4270-253X       0.97      1.00      0.99       206

           accuracy                           0.98       326
          macro avg       0.99      0.97      0.98       326
       weighted avg       0.98      0.98      0.98   

Total missing sample:  0
Text embedding shape:  (413, 100)
Total missing sample:  0
Citation embedding shape:  (0, 0)
Final feature (combined embedding) shape:  (413, 100)
Binary case
                     precision    recall  f1-score   support

0000-0002-1940-6428       0.97      0.98      0.98       219
0000-0002-7074-8087       0.98      0.96      0.97       194

           accuracy                           0.97       413
          macro avg       0.97      0.97      0.97       413
       weighted avg       0.97      0.97      0.97       413

TP:  187 TN:  215 FP:  4 FN:  7
LR Accuracy:  0.9733656174334141
LR F1:  0.9732426303854875
Binary case
                     precision    recall  f1-score   support

0000-0002-1940-6428       0.96      0.97      0.97       219
0000-0002-7074-8087       0.97      0.95      0.96       194

           accuracy                           0.96       413
          macro avg       0.96      0.96      0.96       413
       weighted avg       0.96      

Total missing sample:  0
Text embedding shape:  (536, 100)
Total missing sample:  0
Citation embedding shape:  (0, 0)
Final feature (combined embedding) shape:  (536, 100)
Binary case
                     precision    recall  f1-score   support

0000-0002-4295-6129       0.99      1.00      0.99       423
0000-0002-9639-7940       0.98      0.97      0.98       113

           accuracy                           0.99       536
          macro avg       0.99      0.98      0.99       536
       weighted avg       0.99      0.99      0.99       536

TP:  110 TN:  421 FP:  2 FN:  3
LR Accuracy:  0.9906716417910447
LR F1:  0.9859372950282042
Binary case
                     precision    recall  f1-score   support

0000-0002-4295-6129       0.99      0.99      0.99       423
0000-0002-9639-7940       0.97      0.96      0.97       113

           accuracy                           0.99       536
          macro avg       0.98      0.98      0.98       536
       weighted avg       0.99      

Total missing sample:  0
Text embedding shape:  (470, 100)
Total missing sample:  0
Citation embedding shape:  (0, 0)
Final feature (combined embedding) shape:  (470, 100)
Multi-Class case
                     precision    recall  f1-score   support

0000-0003-1232-5307       0.98      0.98      0.98       124
0000-0003-1835-9436       1.00      1.00      1.00       200
0000-0003-3477-1172       0.99      0.99      0.99       146

           accuracy                           0.99       470
          macro avg       0.99      0.99      0.99       470
       weighted avg       0.99      0.99      0.99       470

[122   0   2   0 200   0   2   0 144]
[[122   0   2]
 [  0 200   0]
 [  2   0 144]]
TP:  [122 200 144] TN:  [344 270 322] FP:  [2 0 2] FN:  [2 0 2]
LR Accuracy:  0.9914893617021276
LR F1:  0.9900574458683163
Multi-Class case
                     precision    recall  f1-score   support

0000-0003-1232-5307       0.95      0.98      0.96       124
0000-0003-1835-9436       1.00   

In [ ]:
print(pp_text_emb)
print(lr_diff_embedding_result)
print(svm_diff_embedding_result)

In [ ]:
# combine n2v with textual embedding result together


In [ ]:
# accuracy
from statistics import mean 
cleaned_svcLinear_accuracy = [x for x in all_svcLinear_accuracy if isinstance(x, float)]
cleaned_lr_accuracy = [x for x in all_LR_accuracy if isinstance(x, float)]
print(len(cleaned_svcLinear_accuracy))
print(len(cleaned_lr_accuracy))
print(mean(cleaned_svcLinear_accuracy))
print(mean(cleaned_lr_accuracy))

In [ ]:
# f1
from statistics import mean 
# remove string from result
cleaned_svcLinear_f1 = [x for x in all_svcLinear_f1 if isinstance(x, float)]
cleaned_lr_f1 = [x for x in all_LR_f1 if isinstance(x, float)]
print(len(cleaned_svcLinear_f1))
print(len(cleaned_lr_f1))
print(mean(cleaned_svcLinear_f1))
print(mean(cleaned_lr_f1))

In [ ]:
%reset

In [ ]:
%who